In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark=SparkSession.builder.appName('File_Validation').getOrCreate()

In [0]:

file_path="dbfs:/FileStore/customers1.csv"  
file_type="csv"
expected_schema=["customer_id","customer_fname","customer_lname","username","password","address","city","state","pincode"]

def duplicate():
    duplicate_rows=df.count()-df.dropDuplicates().count()
    print(f'Number of duplicate rows: {duplicate_rows}')
    df.dropDuplicates()
    return df

def dictinct_count():
    for column in df.columns:
        print(f'{column}:{df.select(column).distinct().count()} distinct values')
    return df

def count_null():
    df.select(*(sum(col(c).isNull().cast("int")).alias(c)for c in df.columns))
    return df

def schema_mismatch():
    if not all(col in df.columns for col in expected_schema):
        print("schema_mismatch")
    else:
        print("No Schema Mismatch")
    return df

def correlation():
    df.select(corr("pincode","address"))
    print(f'show correlation')
    return df

def dependency_chk():
    if df.filter(df.customer_fname.isNull() & df.customer_lname.isNotNull()):
        print(f'consistent data')
    else:
        print(f'inconsistent data')
    return df

def read_file():
    df=spark.read.format(file_type).option("header", "true").load(file_path)
    if df.count()==0: 
        print(file_type + " is empty")
    else:
        print(file_type + " is not empty")
        df.describe().show()
        duplicate()
        dictinct_count()
        count_null()
        schema_mismatch()
        correlation()
        dependency_chk().show()
    return df    

df=read_file()


csv is not empty
+-------+------------------+--------------+--------------+---------+---------+--------------------+----------+-----+------------------+
|summary|       customer_id|customer_fname|customer_lname| username| password|             address|      city|state|           pincode|
+-------+------------------+--------------+--------------+---------+---------+--------------------+----------+-----+------------------+
|  count|             12435|         12435|         12435|    12435|    12435|               12435|     12435|12435|             12435|
|   mean|            6218.0|          null|          null|     null|     null|                null|      null| null|36043.661680739846|
| stddev|3589.8196333520714|          null|          null|     null|     null|                null|      null| null| 37623.00199497027|
|    min|                 1|         Aaron|        Abbott|XXXXXXXXX|XXXXXXXXX|      1 Bright Manor| Aguadilla|   AL|             00603|
|    max|              9999|   